# Archive my tweets

- badges: true
- categories: [code, twitter]


I want to extract all the tweets I've ever written and convert them to small markdown files so they show up as "posts" on this website.

Posts are organized by date, in the traditional blogging format. But so are tweets, kind of? They're listed chronologically anyway. Maybe I could make one post for each day, and then have all the tweets listed on that page. Each one could have its tweet ID as a header, thus having an internal link. Tweets can link to the tweets that precede them, and maybe even backlink to tweets that follow.

It's not quite block references, but as a way of keeping my second brain under my ownership it should work. And this way if anyone wants to cancel me they'll have a convenient search box and permalinks for it. Even if my account gets deleted, my bad takes can stay up.

In [155]:
# Find the last date i ran this
from pathlib import Path
out_dir = Path('../_posts/tweets/')
posts = [o.name for o in out_dir.iterdir()]

last_date = sorted(posts)[-2][:10]

last_date

'2021-08-22'

In [160]:
#hide_output
import twint
import nest_asyncio
nest_asyncio.apply()


c = twint.Config()
c.Username = 'deepfates'
tweets = []
c.Store_object = True
c.Store_object_tweets_list = tweets
c.Since = last_date
c.Hide_output = True
twint.run.Profile(c)

In [163]:
len(tweets)

1506

In [164]:
t = tweets[-1]

In [165]:
t.conversation_id, t.datestamp, t.datetime, t.id, t.likes_count, t.link, t.mentions, t.photos, t.quote_url, t.replies_count, t.reply_to, t.retweet, t.retweet_date, t.retweet_id, t.retweets_count, t.source, t.thumbnail, t.timestamp, t.timezone, t.tweet, t.urls, t.user_id, t.user_id_str, t.user_rt, t.user_rt_id, t.username, t.video

('1429291862033002498',
 '2021-08-22',
 '2021-08-22 00:38:35 MDT',
 1429332153171529729,
 2,
 'https://twitter.com/deepfates/status/1429332153171529729',
 [],
 [],
 '',
 0,
 [{'screen_name': 'reconfigurthing',
   'name': "elias. that's it. ⏳⌛",
   'id': '1158132886983401474'}],
 False,
 '',
 '',
 0,
 '',
 '',
 '00:38:35',
 '-0600',
 '@reconfigurthing Hawkeye (pictured)',
 [],
 3315205122,
 '3315205122',
 '',
 '',
 'deepfates',
 0)

In [166]:
import requests
import shutil

In [167]:
def dl_image(url):
    filename = '../images/' + url.split('/')[-1]
    r = requests.get(url, stream = True)
    if r.status_code == 200:
        r.raw.decode_content = True
        with open(filename,'wb') as f:
            shutil.copyfileobj(r.raw, f)
        return(filename)
    else:
        return(None)
    
# hacky thing uses [1:] to shave the first '.' off the filename
def image_template(filename):
    return(f'![image from twitter](/{filename[1:]})\n')

    
def get_tweet(t):
    if t.photos == []:
        img_md = ''
    else:
        img_list = [dl_image(url) for url in t.photos]
        img_md = '\n'.join([image_template(o) for o in img_list])

    return(f'''
#### <a href = "{t.link}">*{t.timestamp}*</a>

<font size="5">{t.tweet}</font>

{img_md}

🗨️ {t.replies_count} ♺ {t.retweets_count} 🤍  {t.likes_count}   

---
    ''')

def get_md(tweets, date):
    tweets_text = ''.join(t for t in tweets)
    return(f'''---
title: deepfates log {date}
layout: post
toc: true
comments: false
search_exclude: false
categories: [tweets]
---

{tweets_text}
            ''')

In [168]:
from IPython.display import Markdown

In [172]:
yesterday = t.datestamp
y_tweets = [tw for tw in tweets if tw.datestamp == yesterday]
len(y_tweets)

143

In [174]:
Markdown(get_tweet([tw for tw in tweets if tw.datestamp == yesterday][-1]))


#### <a href = "https://twitter.com/deepfates/status/1429332153171529729">*00:38:35*</a>

<font size="5">@reconfigurthing Hawkeye (pictured)</font>



🗨️ 0 ♺ 0 🤍  2   

---
    

In [175]:
y_sorted = sorted(y_tweets, key=lambda x: x.datetime)
# [tweet.tweet for tweet in y_sorted]

Too many replies! Let's limit to just mine for now

In [176]:
y_md = get_md([get_tweet(t) for t in y_sorted if "@" not in t.tweet], yesterday)

In [177]:
with open(f'../_posts/tweets/{yesterday}-tweets.md', 'w') as f:
    print(y_md, file=f)

Okay, that'll do for now. It prints a chronological page of tweets for each day. Linking, video and oter people's tweets will have to come later.

I'll wrap that behavior in a function and pass it my tweets and a set of dates when i have tweeted.

In [178]:
def write_day_page(day, tweets):
    tweets = [tw for tw in tweets if tw.datestamp == day]
    sorted_tweets = sorted(tweets, key=lambda x: x.datetime)
    md = get_md([get_tweet(t) for t in sorted_tweets], day)
    with open(f'../_posts/tweets/{day}-tweets.md', 'w') as f:
        print(md, file=f)

In [179]:
self_tweets = [t for t in tweets if "@" not in t.tweet]

len(self_tweets)

278

In [180]:
days = set([t.datestamp for t in self_tweets])

len(days)

11

In [181]:
from tqdm import tqdm

In [182]:
for day in tqdm(days):
    write_day_page(day, self_tweets)

100%|██████████| 11/11 [00:09<00:00,  1.10it/s]


I would also liek to do analysis to see how often I tweet. And make a big list of links. Maybe next time.

For now you can find these secret tweet archives by searching in the [Explore](/explore) page

In [183]:
days

{'2021-08-22',
 '2021-08-23',
 '2021-08-24',
 '2021-08-25',
 '2021-08-26',
 '2021-08-27',
 '2021-08-28',
 '2021-08-29',
 '2021-08-30',
 '2021-08-31',
 '2021-09-01'}